Monte Carlo theory: Reference book:https://statweb.stanford.edu/~owen/mc/
# Importance Sampling

Our target is to find $\mu$, where $\mu = E[f(x)] = \int_D f(x)p(x) dx$. Simple Monte Carlo method suggests:
\begin{equation}
\mu \approx \frac{1}{N} \sum_{i=1}^N f(x^i)  \qquad x^i \sim p(x)
\end{equation}
However, simple Monte Carlo method may fail when f(x) are nearly zero outside region A and $P(x\in A)$ is small. Intuitively we would like to have more samples from the important region A.

Therefore, we use Importance Sampling:
\begin{align}
\mu & = \int_D f(x) \frac{p(x)}{q(x)}q(x) \quad dx = E_q\Big[f(X)\frac{p(X)}{q(X)}\Big] \\
& \approx \frac{1}{N} \sum_{i=1}^N f(X_i)\frac{p(X_i)}{q(X_i)} \quad X_i \sim q
\end{align}
Assume we can compute f(x), to use the above equation, we have to be able to compute $p(x)/q(x)$. Details about the estimate variance and how to choose $q(x)$ were discussed in details in the reference book.

## self-normalized importance sampling
Sometimes we can only have unnormalized p(x) and q(x), therefore we use:
\begin{equation}
\hat{\mu}_q = \frac{\sum_{i=1}^N f(X_i)w(X_i)}{\sum_{i=1}^N w(X_i)} \quad X_i \sim q
\end{equation}